# Dataset
In this section we will load and pre-process the data

In [1]:
import pandas as pd
from pathlib import Path
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader

In [2]:
# Path definitions
colab_path = '/content/drive/MyDrive/NLP/project/'
local_path = './data/'
data_path = Path(local_path)

train_data_path = data_path / 'MELD_train_efr.json'
val_data_path = data_path / 'MELD_val_efr.json'
test_data_path = data_path / 'MELD_test_efr.json'

In [3]:
train_data_raw = pd.read_json(train_data_path)
val_data_raw = pd.read_json(val_data_path)
test_data_raw = pd.read_json(test_data_path)
train_data_raw.head()

,episode,speakers,emotions,utterances,triggers
0,utterance_0,"[Chandler, The Interviewer, Chandler, The Inte...","[neutral, neutral, neutral, neutral, surprise]",[also I was the point person on my company's t...,"[0.0, 0.0, 0.0, 1.0, 0.0]"
1,utterance_1,"[Chandler, The Interviewer, Chandler, The Inte...","[neutral, neutral, neutral, neutral, surprise,...",[also I was the point person on my company's t...,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]"
2,utterance_2,"[Chandler, The Interviewer, Chandler, The Inte...","[neutral, neutral, neutral, neutral, surprise,...",[also I was the point person on my company's t...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ..."
3,utterance_3,"[Chandler, The Interviewer, Chandler, The Inte...","[neutral, neutral, neutral, neutral, surprise,...",[also I was the point person on my company's t...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,utterance_4,"[Joey, Rachel, Joey, Rachel]","[surprise, sadness, surprise, fear]",[But then who? The waitress I went out with la...,"[0.0, 0.0, 1.0, 0.0]"


## Emotion encoding
The emotions are encoded mapping each of them with a specific value. Positive emotion are assigned to positive values the negatives with negatives

In [4]:
emotions = train_data_raw['emotions'].explode().unique()
print(emotions)

['neutral' 'surprise' 'fear' 'sadness' 'joy' 'disgust' 'anger']


In [5]:

# Encode emotions
encoder = LabelEncoder()
encoder.fit(emotions)
print("Mapping:" + str(dict(zip(encoder.classes_, encoder.transform(encoder.classes_)))))

Mapping:{'anger': 0, 'disgust': 1, 'fear': 2, 'joy': 3, 'neutral': 4, 'sadness': 5, 'surprise': 6}


In [6]:
def pre_process(data):
    tmp = data.copy()
    tmp['emotions'] = data['emotions'].apply(lambda x: encoder.transform(x))
    return tmp

In [7]:
train_data = pre_process(train_data_raw)
val_data = pre_process(val_data_raw)
test_data = pre_process(test_data_raw)

In [8]:
# Show the first row
train_data

,episode,speakers,emotions,utterances,triggers
0,utterance_0,"[Chandler, The Interviewer, Chandler, The Inte...","[4, 4, 4, 4, 6]",[also I was the point person on my company's t...,"[0.0, 0.0, 0.0, 1.0, 0.0]"
1,utterance_1,"[Chandler, The Interviewer, Chandler, The Inte...","[4, 4, 4, 4, 6, 4, 4]",[also I was the point person on my company's t...,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]"
2,utterance_2,"[Chandler, The Interviewer, Chandler, The Inte...","[4, 4, 4, 4, 6, 4, 4, 4, 4, 4, 2]",[also I was the point person on my company's t...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ..."
3,utterance_3,"[Chandler, The Interviewer, Chandler, The Inte...","[4, 4, 4, 4, 6, 4, 4, 4, 4, 4, 2, 4, 6]",[also I was the point person on my company's t...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,utterance_4,"[Joey, Rachel, Joey, Rachel]","[6, 5, 6, 2]",[But then who? The waitress I went out with la...,"[0.0, 0.0, 1.0, 0.0]"
...,...,...,...,...,...
3995,utterance_3995,"[Chandler, All, Monica, Chandler, Ross, Chandl...","[4, 3, 4, 4, 6, 1, 4, 1, 6, 4, 4, 1]","[Hey., Hey!, So how was Joan?, I broke up with...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3996,utterance_3996,"[Chandler, All, Monica, Chandler, Ross, Chandl...","[4, 3, 4, 4, 6, 1, 4, 1, 6, 4, 4, 1, 1, 4]","[Hey., Hey!, So how was Joan?, I broke up with...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3997,utterance_3997,"[Chandler, All, Monica, Chandler, Ross, Chandl...","[4, 3, 4, 4, 6, 1, 4, 1, 6, 4, 4, 1, 1, 4, 4]","[Hey., Hey!, So how was Joan?, I broke up with...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3998,utterance_3998,"[Chandler, All, Monica, Chandler, Ross, Chandl...","[4, 3, 4, 4, 6, 1, 4, 1, 6, 4, 4, 1, 1, 4, 4, 6]","[Hey., Hey!, So how was Joan?, I broke up with...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [9]:
class UtteranceDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data.index)

    def __getitem__(self, idx):
        return self.data.iloc[idx]

In [10]:
%load_ext autoreload
%autoreload 2

In [11]:
from baselines import BertBaseline 

In [12]:
model = BertBaseline()
train_dataset = UtteranceDataset(train_data)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)

In [17]:
for batch in train_loader:
    print(batch.shape)
    model.forward(batch, epochs=1)
    break

TypeError: Caught TypeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/elements72/.local/lib/python3.10/site-packages/torch/utils/data/_utils/worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/elements72/.local/lib/python3.10/site-packages/torch/utils/data/_utils/fetch.py", line 54, in fetch
    return self.collate_fn(data)
  File "/home/elements72/.local/lib/python3.10/site-packages/torch/utils/data/_utils/collate.py", line 265, in default_collate
    return collate(batch, collate_fn_map=default_collate_fn_map)
  File "/home/elements72/.local/lib/python3.10/site-packages/torch/utils/data/_utils/collate.py", line 150, in collate
    raise TypeError(default_collate_err_msg_format.format(elem_type))
TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'pandas.core.series.Series'>
